In [318]:
import pandas as pd
import numpy as np
%matplotlib inline

from datetime import datetime
from datetime import timedelta

from sklearn import tree
from imblearn.over_sampling import SMOTE
from sklearn.metrics import r2_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier,GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV


In [239]:
# Importing the dataset
train = pd.read_csv('train.csv', dtype=dtypes)
test = pd.read_csv('test.csv', dtype=dtypes)

In [240]:
target = pd.read_csv('train_target.csv')
train_Y = target['target']
train_num = train_Y.shape[0]
target.head()

,index,sequence_id,target
0,3,140366A0464161,0
1,4,140366A0464178,0
2,5,140366A0464189,0
3,6,140366A0464473,0
4,7,140366A0464890,1


In [241]:
test.head()

,sequence_id,ordered_date,ordered_quantity,site_id,account,warehouse,item,item_description,canceled_quantity,item_type,...,state,province,country,su_warehouse,su_market_segment,sales_territory_1,sales_territory_2,sales_territory_3,sales_territory_4,sales_territory_5
0,140365A0464178,2017-01-08,1.0,Site 15,Acc23,258,Item 50,"ACETYLENE, C2H2, 8, (~1.1M3 / ~38.85SCF), CGA-...",0,CG,...,State 8,Province 8,Country 0,258,CN0300,Sales Territory 1.2,Sales Territory 2.17,Sales Territory 3.69,Sales Territory 4.9,Sales Territory 5.0
1,140365A0464178,2017-01-15,0.0,Site 15,Acc23,258,Item 50,"ACETYLENE, C2H2, 8, (~1.1M3 / ~38.85SCF), CGA-...",0,CG,...,State 8,Province 8,Country 0,258,CN0300,Sales Territory 1.2,Sales Territory 2.17,Sales Territory 3.69,Sales Territory 4.9,Sales Territory 5.0
2,140365A0464178,2017-01-22,0.0,Site 15,Acc23,258,Item 50,"ACETYLENE, C2H2, 8, (~1.1M3 / ~38.85SCF), CGA-...",0,CG,...,State 8,Province 8,Country 0,258,CN0300,Sales Territory 1.2,Sales Territory 2.17,Sales Territory 3.69,Sales Territory 4.9,Sales Territory 5.0
3,140365A0464178,2017-01-29,0.0,Site 15,Acc23,258,Item 50,"ACETYLENE, C2H2, 8, (~1.1M3 / ~38.85SCF), CGA-...",0,CG,...,State 8,Province 8,Country 0,258,CN0300,Sales Territory 1.2,Sales Territory 2.17,Sales Territory 3.69,Sales Territory 4.9,Sales Territory 5.0
4,140365A0464178,2017-02-05,0.0,Site 15,Acc23,258,Item 50,"ACETYLENE, C2H2, 8, (~1.1M3 / ~38.85SCF), CGA-...",0,CG,...,State 8,Province 8,Country 0,258,CN0300,Sales Territory 1.2,Sales Territory 2.17,Sales Territory 3.69,Sales Territory 4.9,Sales Territory 5.0


In [245]:
df_all['sequence_id'].nunique()

19564

In [244]:
df_all = pd.concat([train,test])
df_all.head()

,sequence_id,ordered_date,ordered_quantity,site_id,account,warehouse,item,item_description,canceled_quantity,item_type,...,state,province,country,su_warehouse,su_market_segment,sales_territory_1,sales_territory_2,sales_territory_3,sales_territory_4,sales_territory_5
0,140366A0464161,2017-01-29,1.0,Site 16,Acc24,944,Item 42,"ACETYLENE, C2H2, 2, (~0.28M3 / ~9.89SCF), CGA-...",0,CG,...,State 11,Province 11,Country 0,944,CN0200,Sales Territory 1.4,Sales Territory 2.20,Sales Territory 3.70,Sales Territory 4.11,Sales Territory 5.0
1,140366A0464161,2017-02-05,0.0,Site 16,Acc24,944,Item 42,"ACETYLENE, C2H2, 2, (~0.28M3 / ~9.89SCF), CGA-...",0,CG,...,State 11,Province 11,Country 0,944,CN0200,Sales Territory 1.4,Sales Territory 2.20,Sales Territory 3.70,Sales Territory 4.11,Sales Territory 5.0
2,140366A0464161,2017-02-12,0.0,Site 16,Acc24,944,Item 42,"ACETYLENE, C2H2, 2, (~0.28M3 / ~9.89SCF), CGA-...",0,CG,...,State 11,Province 11,Country 0,944,CN0200,Sales Territory 1.4,Sales Territory 2.20,Sales Territory 3.70,Sales Territory 4.11,Sales Territory 5.0
3,140366A0464161,2017-02-19,0.0,Site 16,Acc24,944,Item 42,"ACETYLENE, C2H2, 2, (~0.28M3 / ~9.89SCF), CGA-...",0,CG,...,State 11,Province 11,Country 0,944,CN0200,Sales Territory 1.4,Sales Territory 2.20,Sales Territory 3.70,Sales Territory 4.11,Sales Territory 5.0
4,140366A0464161,2017-02-26,0.0,Site 16,Acc24,944,Item 42,"ACETYLENE, C2H2, 2, (~0.28M3 / ~9.89SCF), CGA-...",0,CG,...,State 11,Province 11,Country 0,944,CN0200,Sales Territory 1.4,Sales Territory 2.20,Sales Territory 3.70,Sales Territory 4.11,Sales Territory 5.0


In [246]:
df_all = df_all.drop(['business_line', 'customer_type', 'sales_territory_5'], axis=1)
df_all.head()

,sequence_id,ordered_date,ordered_quantity,site_id,account,warehouse,item,item_description,canceled_quantity,item_type,...,postal_code,state,province,country,su_warehouse,su_market_segment,sales_territory_1,sales_territory_2,sales_territory_3,sales_territory_4
0,140366A0464161,2017-01-29,1.0,Site 16,Acc24,944,Item 42,"ACETYLENE, C2H2, 2, (~0.28M3 / ~9.89SCF), CGA-...",0,CG,...,PS 1241,State 11,Province 11,Country 0,944,CN0200,Sales Territory 1.4,Sales Territory 2.20,Sales Territory 3.70,Sales Territory 4.11
1,140366A0464161,2017-02-05,0.0,Site 16,Acc24,944,Item 42,"ACETYLENE, C2H2, 2, (~0.28M3 / ~9.89SCF), CGA-...",0,CG,...,PS 1241,State 11,Province 11,Country 0,944,CN0200,Sales Territory 1.4,Sales Territory 2.20,Sales Territory 3.70,Sales Territory 4.11
2,140366A0464161,2017-02-12,0.0,Site 16,Acc24,944,Item 42,"ACETYLENE, C2H2, 2, (~0.28M3 / ~9.89SCF), CGA-...",0,CG,...,PS 1241,State 11,Province 11,Country 0,944,CN0200,Sales Territory 1.4,Sales Territory 2.20,Sales Territory 3.70,Sales Territory 4.11
3,140366A0464161,2017-02-19,0.0,Site 16,Acc24,944,Item 42,"ACETYLENE, C2H2, 2, (~0.28M3 / ~9.89SCF), CGA-...",0,CG,...,PS 1241,State 11,Province 11,Country 0,944,CN0200,Sales Territory 1.4,Sales Territory 2.20,Sales Territory 3.70,Sales Territory 4.11
4,140366A0464161,2017-02-26,0.0,Site 16,Acc24,944,Item 42,"ACETYLENE, C2H2, 2, (~0.28M3 / ~9.89SCF), CGA-...",0,CG,...,PS 1241,State 11,Province 11,Country 0,944,CN0200,Sales Territory 1.4,Sales Territory 2.20,Sales Territory 3.70,Sales Territory 4.11


### Feature Engineering

#### date processing 

In [247]:
df_all['Date'] = pd.to_datetime(df_all['ordered_date'])
df_all['wom'] = (df_all['Date'].dt.day-1)//7+1

In [252]:
#df_all[df_all['same_warehouse']!=1]
df_all[df_all['sequence_id']== '140366A0464161']

,sequence_id,ordered_date,ordered_quantity,site_id,account,warehouse,item,item_description,canceled_quantity,item_type,...,sales_territory_3,sales_territory_4,Date,wom,days_diff,week_diff,month_diff,week_number,month,same_warehouse
0,140366A0464161,2017-01-29,1.0,Site 16,Acc24,944,Item 42,"ACETYLENE, C2H2, 2, (~0.28M3 / ~9.89SCF), CGA-...",0,CG,...,Sales Territory 3.70,Sales Territory 4.11,2017-01-29,5,777,111,25,4,1,1.0
1,140366A0464161,2017-02-05,0.0,Site 16,Acc24,944,Item 42,"ACETYLENE, C2H2, 2, (~0.28M3 / ~9.89SCF), CGA-...",0,CG,...,Sales Territory 3.70,Sales Territory 4.11,2017-02-05,1,770,110,25,5,2,1.0
2,140366A0464161,2017-02-12,0.0,Site 16,Acc24,944,Item 42,"ACETYLENE, C2H2, 2, (~0.28M3 / ~9.89SCF), CGA-...",0,CG,...,Sales Territory 3.70,Sales Territory 4.11,2017-02-12,2,763,109,25,6,2,1.0
3,140366A0464161,2017-02-19,0.0,Site 16,Acc24,944,Item 42,"ACETYLENE, C2H2, 2, (~0.28M3 / ~9.89SCF), CGA-...",0,CG,...,Sales Territory 3.70,Sales Territory 4.11,2017-02-19,3,756,108,25,7,2,1.0
4,140366A0464161,2017-02-26,0.0,Site 16,Acc24,944,Item 42,"ACETYLENE, C2H2, 2, (~0.28M3 / ~9.89SCF), CGA-...",0,CG,...,Sales Territory 3.70,Sales Territory 4.11,2017-02-26,4,749,107,24,8,2,1.0
5,140366A0464161,2017-03-05,0.0,Site 16,Acc24,944,Item 42,"ACETYLENE, C2H2, 2, (~0.28M3 / ~9.89SCF), CGA-...",0,CG,...,Sales Territory 3.70,Sales Territory 4.11,2017-03-05,1,742,106,24,9,3,1.0
6,140366A0464161,2017-03-12,1.0,Site 16,Acc24,944,Item 42,"ACETYLENE, C2H2, 2, (~0.28M3 / ~9.89SCF), CGA-...",0,CG,...,Sales Territory 3.70,Sales Territory 4.11,2017-03-12,2,735,105,24,10,3,1.0
7,140366A0464161,2017-03-19,0.0,Site 16,Acc24,944,Item 42,"ACETYLENE, C2H2, 2, (~0.28M3 / ~9.89SCF), CGA-...",0,CG,...,Sales Territory 3.70,Sales Territory 4.11,2017-03-19,3,728,104,24,11,3,1.0
8,140366A0464161,2017-03-26,0.0,Site 16,Acc24,944,Item 42,"ACETYLENE, C2H2, 2, (~0.28M3 / ~9.89SCF), CGA-...",0,CG,...,Sales Territory 3.70,Sales Territory 4.11,2017-03-26,4,721,103,24,12,3,1.0
9,140366A0464161,2017-04-02,0.0,Site 16,Acc24,944,Item 42,"ACETYLENE, C2H2, 2, (~0.28M3 / ~9.89SCF), CGA-...",0,CG,...,Sales Territory 3.70,Sales Territory 4.11,2017-04-02,1,714,102,23,13,4,1.0


In [248]:
fixed_date = datetime(2019,3,17)
df_all['days_diff'] = (fixed_date- df_all['Date']).apply(lambda x: x.days)
df_all['week_diff'] = df_all['days_diff']//7
df_all['month_diff'] = df_all['days_diff']//30

In [249]:
df_all['week_number'] = df_all['Date'].dt.week
df_all['month'] = df_all['Date'].dt.month

### Ordered Quantity

In [119]:
#處理last n week order quantity
bin_cut = np.linspace(1, 121, num=21)
test = pd.cut(df_all['week_diff'], bins = bin_cut) 
#df_all.pivot_table(index='sequence_id', columns=test, aggfunc={'ordered_quantity':sum, 'canceled_quantity':sum})

df_test = df_all.pivot_table(index='sequence_id', columns=test, aggfunc={'ordered_quantity':sum})
df_test.replace(np.nan, 0, inplace=True)
df_test.columns = df_test.columns.droplevel()
df_test.columns = [
'last_1_6_wk_orderQ','last_7_12_wk_orderQ','last_13_18_wk_orderQ',  'last_19_24_wk_orderQ','last_25_30_wk_orderQ', 'last_31_36_wk_orderQ', 'last_37_42_wk_orderQ',
'last_43_48_wk_orderQ','last_49_54_wk_orderQ','last_55_60_wk_orderQ','last_61_66_wk_orderQ','last_67_72_wk_orderQ','last_73_78_wk_orderQ','last_79_84_wk_orderQ',
'last_85_90_wk_orderQ','last_91_96_wk_orderQ','last_97_102_wk_orderQ','last_103_108_wk_orderQ', 'last_109_114_wk_orderQ'
]
df_test.reset_index(inplace=True)

#df_test['sequence_id'] = df_test.index.tolist() 

In [140]:
#處理last n month order quantity

#test = pd.cut(df_all['month_diff'], bins = bin_cut) 
#df_all.pivot_table(index='sequence_id', columns=test, aggfunc={'ordered_quantity':sum, 'canceled_quantity':sum})
df_mo_1 = df_all.pivot_table(index='sequence_id', columns=['month_diff'], aggfunc={'ordered_quantity':sum})
df_mo_1.replace(np.nan, 0, inplace=True)
df_mo_1.columns = df_mo_1.columns.droplevel()
df_mo_1.columns = [
'last_0_mo_orderQ','last_1_mo_orderQ','last_2_mo_orderQ',  'last_3_mo_orderQ','last_4_mo_orderQ', 'last_5_mo_orderQ', 'last_6_mo_orderQ',
'last_7_mo_orderQ','last_8_mo_orderQ','last_9_mo_orderQ','last_10_mo_orderQ','last_11_mo_orderQ','last_12_mo_orderQ','last_13_mo_orderQ',
'last_14_mo_orderQ','last_15_mo_orderQ','last_16_mo_orderQ','last_17_mo_orderQ', 'last_18_mo_orderQ','last_19_mo_orderQ','last_20_mo_orderQ',
'last_21_mo_orderQ','last_22_mo_orderQ','last_23_mo_orderQ','last_24_mo_orderQ','last_25_mo_orderQ', 'last_26_mo_orderQ'
] 
df_mo_1.reset_index(inplace=True)

In [ ]:
#處理last n~n+3 month order quantity
test = pd.cut(df_all['month_diff'], bins = [0,3,6,9,12,15,18,21,24,27]) 
df_mo_3 = df_all.pivot_table(index='sequence_id', columns=test, aggfunc={'ordered_quantity':sum})
df_mo_3.replace(np.nan, 0, inplace=True)
df_mo_3.columns = df_mo_3.columns.droplevel()
df_mo_3.columns = [
'last_0_3_mo_orderQ','last_3_6_mo_orderQ','last_6_9_mo_orderQ',  'last_9_12_mo_orderQ','last_12_15_mo_orderQ', 'last_15_18_mo_orderQ',
'last_18_21_mo_orderQ','last_21_24_mo_orderQ','last_24_27_mo_orderQ'
] df_mo_3.reset_index(inplace=True)

In [163]:
#處理last n~n+3 month order quantity slope
df_mo_3['last_3_mo_orderQ_slope_3'] = df_mo_3['last_0_3_mo_orderQ']/df_mo_3['last_3_6_mo_orderQ']
df_mo_3['last_6_mo_orderQ_slope_3'] = df_mo_3['last_3_6_mo_orderQ']/df_mo_3['last_6_9_mo_orderQ']
df_mo_3['last_9_mo_orderQ_slope_3'] = df_mo_3['last_6_9_mo_orderQ']/df_mo_3['last_9_12_mo_orderQ']
df_mo_3['last_12_mo_orderQ_slope_3'] = df_mo_3['last_9_12_mo_orderQ']/df_mo_3['last_12_15_mo_orderQ']
df_mo_3['last_15_mo_orderQ_slope_3'] = df_mo_3['last_12_15_mo_orderQ']/df_mo_3['last_15_18_mo_orderQ']
df_mo_3['last_18_mo_orderQ_slope_3'] = df_mo_3['last_15_18_mo_orderQ']/df_mo_3['last_18_21_mo_orderQ']
df_mo_3['last_21_mo_orderQ_slope_3'] = df_mo_3['last_18_21_mo_orderQ']/df_mo_3['last_21_24_mo_orderQ']
df_mo_3['last_24_mo_orderQ_slope_3'] = df_mo_3['last_21_24_mo_orderQ']/df_mo_3['last_24_27_mo_orderQ']

df_mo_3.replace(np.inf, 1000, inplace=True)

In [153]:
#處理last n~n+6 month order quantity
test = pd.cut(df_all['month_diff'], bins = [0,6,12,18,24,30]) 
df_mo_6 = df_all.pivot_table(index='sequence_id', columns=test, aggfunc={'ordered_quantity':sum})
df_mo_6.replace(np.nan, 0, inplace=True)
df_mo_6.columns = df_mo_6.columns.droplevel()
df_mo_6.columns = ['last_0_6_mo_orderQ','last_6_12_mo_orderQ','last_12_18_mo_orderQ',  'last_18_24_mo_orderQ','last_24_30_mo_orderQ'] 
df_mo_6.reset_index(inplace=True)

In [160]:
#處理last n~n+6 month order quantity slope
df_mo_6['last_6_mo_orderQ_slope_6'] = df_mo_6['last_0_6_mo_orderQ']/df_mo_6['last_6_12_mo_orderQ']
df_mo_6['last_12_mo_orderQ_slope_6'] = df_mo_6['last_6_12_mo_orderQ']/df_mo_6['last_12_18_mo_orderQ']
df_mo_6['last_18_mo_orderQ_slope_6'] = df_mo_6['last_12_18_mo_orderQ']/df_mo_6['last_18_24_mo_orderQ']
df_mo_6['last_24_mo_orderQ_slope_6'] = df_mo_6['last_18_24_mo_orderQ']/df_mo_6['last_24_30_mo_orderQ']
df_mo_6.replace(np.inf, 1000, inplace=True)

In [201]:
df_mo_3.head(2)

,sequence_id,last_0_3_mo_orderQ,last_3_6_mo_orderQ,last_6_9_mo_orderQ,last_9_12_mo_orderQ,last_12_15_mo_orderQ,last_15_18_mo_orderQ,last_18_21_mo_orderQ,last_21_24_mo_orderQ,last_24_27_mo_orderQ,last_3_mo_orderQ_slope_3,last_6_mo_orderQ_slope_3,last_9_mo_orderQ_slope_3,last_12_mo_orderQ_slope_3,last_15_mo_orderQ_slope_3,last_18_mo_orderQ_slope_3,last_21_mo_orderQ_slope_3,last_24_mo_orderQ_slope_3
0,140365A0464178,0.0,3.0,2.0,2.0,1.0,1.0,2.0,3.0,1.0,0.0,1.50,1.0,2.000000,1.0,0.5,0.666667,3.0
1,140365A0464474,3.0,1.0,4.0,4.0,3.0,3.0,2.0,4.0,0.0,3.0,0.25,1.0,1.333333,1.0,1.5,0.500000,1000.0


In [168]:
#處理nth month order quantity
df_n_month = df_all.pivot_table(index='sequence_id', columns='month', aggfunc={'ordered_quantity':np.mean})
df_n_month.replace(np.nan, 0, inplace=True)
df_n_month.columns = df_n_month.columns.droplevel()
df_n_month.columns = ['1_mo_orderQ','2_mo_orderQ','3_mo_orderQ',  '4_mo_orderQ','5_mo_orderQ','6_mo_orderQ',
                      '7_mo_orderQ','8_mo_orderQ','9_mo_orderQ','10_mo_orderQ','11_mo_orderQ','12_mo_orderQ'] 
df_n_month.reset_index(inplace=True)

In [174]:
#處理nth month order quantity slope
df_n_month['1_mo_orderQ_slope'] = df_n_month['1_mo_orderQ']/df_n_month['12_mo_orderQ']
df_n_month['2_mo_orderQ_slope'] = df_n_month['2_mo_orderQ']/df_n_month['1_mo_orderQ']
df_n_month['3_mo_orderQ_slope'] = df_n_month['3_mo_orderQ']/df_n_month['2_mo_orderQ']
df_n_month['4_mo_orderQ_slope'] = df_n_month['4_mo_orderQ']/df_n_month['3_mo_orderQ']
df_n_month['5_mo_orderQ_slope'] = df_n_month['5_mo_orderQ']/df_n_month['4_mo_orderQ']
df_n_month['6_mo_orderQ_slope'] = df_n_month['6_mo_orderQ']/df_n_month['5_mo_orderQ']
df_n_month['7_mo_orderQ_slope'] = df_n_month['7_mo_orderQ']/df_n_month['6_mo_orderQ']
df_n_month['8_mo_orderQ_slope'] = df_n_month['8_mo_orderQ']/df_n_month['7_mo_orderQ']
df_n_month['9_mo_orderQ_slope'] = df_n_month['9_mo_orderQ']/df_n_month['8_mo_orderQ']
df_n_month['10_mo_orderQ_slope'] = df_n_month['10_mo_orderQ']/df_n_month['9_mo_orderQ']
df_n_month['11_mo_orderQ_slope'] = df_n_month['11_mo_orderQ']/df_n_month['10_mo_orderQ']
df_n_month['12_mo_orderQ_slope'] = df_n_month['12_mo_orderQ']/df_n_month['11_mo_orderQ']
df_n_month.replace(np.inf, 1000, inplace=True)

In [171]:
#處理quarter order quantity
test = pd.cut(df_all['month'], bins = [1,4,7,10,13]) 
df_n_quarter = df_all.pivot_table(index='sequence_id', columns=test, aggfunc={'ordered_quantity':sum})
df_n_quarter.replace(np.nan, 0, inplace=True)
df_n_quarter.columns = df_n_quarter.columns.droplevel()
df_n_quarter.columns = ['Q1_orderQ','Q2_orderQ','Q3_orderQ','Q4_orderQ'] 
df_n_quarter.reset_index(inplace=True)

In [172]:
#處理quarter order quantity slope
df_n_quarter['Q1_orderQ_slope'] = df_n_quarter['Q1_orderQ']/df_n_quarter['Q4_orderQ']
df_n_quarter['Q2_orderQ_slope'] = df_n_quarter['Q2_orderQ']/df_n_quarter['Q1_orderQ']
df_n_quarter['Q3_orderQ_slope'] = df_n_quarter['Q3_orderQ']/df_n_quarter['Q2_orderQ']
df_n_quarter['Q4_orderQ_slope'] = df_n_quarter['Q4_orderQ']/df_n_quarter['Q3_orderQ']
df_n_quarter.replace(np.inf, 1000, inplace=True)

In [178]:
#處理nth week order quantity
df_n_week = df_all.pivot_table(index='sequence_id', columns='wom', aggfunc={'ordered_quantity':np.mean})
df_n_week.replace(np.nan, 0, inplace=True)
df_n_week.columns = df_n_week.columns.droplevel()
df_n_week.columns = ['1_wk_orderQ','2_wk_orderQ','3_wk_orderQ',  '4_wk_orderQ','5_wk_orderQ'] 
df_n_week.reset_index(inplace=True)

In [179]:
#處理nth week order quantity slope
df_n_week['1_wk_orderQ_slope'] = df_n_week['1_wk_orderQ']/df_n_week['5_wk_orderQ']
df_n_week['2_wk_orderQ_slope'] = df_n_week['2_wk_orderQ']/df_n_week['1_wk_orderQ']
df_n_week['3_wk_orderQ_slope'] = df_n_week['3_wk_orderQ']/df_n_week['2_wk_orderQ']
df_n_week['4_wk_orderQ_slope'] = df_n_week['4_wk_orderQ']/df_n_week['3_wk_orderQ']
df_n_week['5_wk_orderQ_slope'] = df_n_week['5_wk_orderQ']/df_n_week['4_wk_orderQ']
df_n_week.replace(np.inf, 1000, inplace=True)
df_n_week.replace(np.nan, 0, inplace=True)

### Canceled Quantity

In [196]:
#處理last n~n+3 month cancel quantity
test = pd.cut(df_all['month_diff'], bins = [0,3,6,9,12,15,18,21,24,27]) 
df_mo_3_cancel = df_all.pivot_table(index='sequence_id', columns=test, aggfunc={'canceled_quantity':sum})
df_mo_3_cancel.replace(np.nan, 0, inplace=True)
df_mo_3_cancel.columns = df_mo_3_cancel.columns.droplevel()
df_mo_3_cancel.columns = [
'last_0_3_mo_cancelQ','last_3_6_mo_cancelQ','last_6_9_mo_cancelQ',  'last_9_12_mo_cancelQ','last_12_15_mo_cancelQ', 'last_15_18_mo_cancelQ',
'last_18_21_mo_cancelQ','last_21_24_mo_cancelQ','last_24_27_mo_cancelQ']
df_mo_3_cancel.reset_index(inplace=True)

#處理last n~n+3 month cancel quantity slope
df_mo_3_cancel['last_3_mo_cancelQ_slope_3'] = df_mo_3_cancel['last_0_3_mo_cancelQ']/df_mo_3_cancel['last_3_6_mo_cancelQ']
df_mo_3_cancel['last_6_mo_cancelQ_slope_3'] = df_mo_3_cancel['last_3_6_mo_cancelQ']/df_mo_3_cancel['last_6_9_mo_cancelQ']
df_mo_3_cancel['last_9_mo_cancelQ_slope_3'] = df_mo_3_cancel['last_6_9_mo_cancelQ']/df_mo_3_cancel['last_9_12_mo_cancelQ']
df_mo_3_cancel['last_12_mo_cancelQ_slope_3'] = df_mo_3_cancel['last_9_12_mo_cancelQ']/df_mo_3_cancel['last_12_15_mo_cancelQ']
df_mo_3_cancel['last_15_mo_cancelQ_slope_3'] = df_mo_3_cancel['last_12_15_mo_cancelQ']/df_mo_3_cancel['last_15_18_mo_cancelQ']
df_mo_3_cancel['last_18_mo_cancelQ_slope_3'] = df_mo_3_cancel['last_15_18_mo_cancelQ']/df_mo_3_cancel['last_18_21_mo_cancelQ']
df_mo_3_cancel['last_21_mo_cancelQ_slope_3'] = df_mo_3_cancel['last_18_21_mo_cancelQ']/df_mo_3_cancel['last_21_24_mo_cancelQ']
df_mo_3_cancel['last_24_mo_cancelQ_slope_3'] = df_mo_3_cancel['last_21_24_mo_cancelQ']/df_mo_3_cancel['last_24_27_mo_cancelQ']

df_mo_3_cancel.replace(np.inf, 1000, inplace=True)
df_mo_3_cancel.replace(np.nan, 0, inplace=True)

In [193]:
df_total_Q = df_all.groupby("sequence_id")[['ordered_quantity','canceled_quantity','net_volume']].sum().reset_index()
df_total_Q['canceled_ratio'] = df_total_Q['canceled_quantity']/df_total_Q['ordered_quantity']
df_total_Q['isOrdered'] = df_total_Q['ordered_quantity'].apply(lambda x: 1 if x >0 else 0)
df_total_Q['isCanceled'] = df_total_Q['canceled_quantity'].apply(lambda x: 1 if x >0 else 0)
df_total_Q.replace(np.inf, 1000, inplace=True)
df_total_Q.replace(np.nan, 0, inplace=True)

### Categorical features

In [250]:
# same warehouse or not
df_all.loc[df_all.su_warehouse == df_all.warehouse, 'same_warehouse'] = 1
df_all.loc[df_all.su_warehouse != df_all.warehouse, 'same_warehouse'] = 0

In [272]:
df_unique = df_all[['sequence_id','site_id','item','item_type','item_ownership','product_line','customer_class_code','city','postal_code','state','province','su_market_segment','sales_territory_1','sales_territory_2','sales_territory_3','sales_territory_4']].drop_duplicates()
df_unique.reset_index(inplace=True)
df_unique.drop(['index'], axis=1,inplace=True)

In [273]:
df_unique.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19564 entries, 0 to 19563
Data columns (total 16 columns):
sequence_id            19564 non-null object
site_id                19564 non-null object
item                   19564 non-null object
item_type              19564 non-null object
item_ownership         19563 non-null object
product_line           19557 non-null object
customer_class_code    19564 non-null object
city                   19564 non-null object
postal_code            19564 non-null object
state                  19564 non-null object
province               19564 non-null object
su_market_segment      19564 non-null object
sales_territory_1      19564 non-null object
sales_territory_2      19564 non-null object
sales_territory_3      19564 non-null object
sales_territory_4      19564 non-null object
dtypes: object(16)
memory usage: 2.4+ MB


In [ ]:
# df_test
#df_mo_1
#df_mo_3
#df_mo_6
#df_n_month
#df_n_quarter
#df_n_week
#df_mo_3_cancel
#df_total_Q

# df_unique
pd.concat([df_test,df_mo_1,df_mo_3,df_mo_6,df_n_month,df_n_quarter,df_n_week,df_mo_3_cancel,df_total_Q,df_temp],axis=1).head()

### Label Encoding

In [275]:
df_unique.fillna('None',inplace=True)

In [276]:
# label encoding
df_temp = pd.DataFrame()
for c in df_unique.columns:
    if c != 'sequence_id':
        df_temp[c] = LabelEncoder().fit_transform(df_unique[c])
    else:
        df_temp[c] = df_unique[c]

### concat all df

In [298]:
df_final.shape

(19564, 153)

In [292]:
df_final = pd.merge(df_temp,df_test, on='sequence_id')
df_final = pd.merge(df_final,df_mo_1, on='sequence_id')
df_final = pd.merge(df_final,df_mo_3, on='sequence_id')
df_final = pd.merge(df_final,df_mo_6, on='sequence_id')
df_final = pd.merge(df_final,df_n_month, on='sequence_id')
df_final = pd.merge(df_final,df_n_quarter, on='sequence_id')
df_final = pd.merge(df_final,df_n_week, on='sequence_id')
df_final = pd.merge(df_final,df_mo_3_cancel, on='sequence_id')
df_final = pd.merge(df_final,df_total_Q, on='sequence_id')

In [299]:
df_final.iloc[1]

sequence_id                   140366A0464178
site_id                                  668
item                                     446
item_type                                  1
item_ownership                             2
product_line                               0
customer_class_code                        1
city                                     829
postal_code                              271
state                                      3
province                                   3
su_market_segment                         12
sales_territory_1                          4
sales_territory_2                         13
sales_territory_3                         68
sales_territory_4                          3
last_1_6_wk_orderQ                         1
last_7_12_wk_orderQ                        1
last_13_18_wk_orderQ                       3
last_19_24_wk_orderQ                       1
last_25_30_wk_orderQ                       2
last_31_36_wk_orderQ                       2
last_37_42

In [ ]:
train_x_all = df_final[:train_num]
test_x_all = df_final[train_num+1:]

## Model

In [309]:
x_columns = train_x_all.drop(['sequence_id'],axis =1).columns
#x_columns = [
#'last_0_3_mo_orderQ' 
#]

In [314]:
x = train_x_all.loc[:, x_colmuns].values
y = train_Y.values


In [326]:
# random sampling: 25% train 75% validate
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1,stratify=y)

# 运用SMOTE算法实现训练数据集的平衡
over_samples = SMOTE(random_state=1)
over_samples_x,over_samples_y = over_samples.fit_sample(x_train, y_train)
print(pd.Series(y_train).value_counts()/len(y_train))
print(pd.Series(over_samples_y).value_counts()/len(over_samples_y))

0    0.701918
1    0.298082
dtype: float64
1    0.5
0    0.5
dtype: float64


### GradientBoostingClassifier

In [327]:
# Baseline 
gbc = GradientBoostingClassifier(random_state=1) 
gbc.fit(x_train, y_train)
gbc_y_predict = gbc.predict(x_test)

In [320]:
#evaluation
print("confusion matrix：\n", confusion_matrix(y_test,gbc_y_predict))
print("metrics：\n", classification_report(y_test,gbc_y_predict))
print("feature importance:\n",pd.Series(gbc.feature_importances_,x_columns).sort_values(ascending=False))

confusion matrix：
 [[2185  218]
 [ 569  452]]
metrics：
               precision    recall  f1-score   support

           0       0.79      0.91      0.85      2403
           1       0.67      0.44      0.53      1021

    accuracy                           0.77      3424
   macro avg       0.73      0.68      0.69      3424
weighted avg       0.76      0.77      0.75      3424

feature importance:
 last_0_3_mo_orderQ            0.237701
last_1_6_wk_orderQ            0.171332
last_2_mo_orderQ              0.065993
last_1_mo_orderQ              0.060502
last_0_6_mo_orderQ            0.056257
last_7_12_wk_orderQ           0.041184
last_4_mo_orderQ              0.035906
net_volume                    0.019004
3_mo_orderQ_slope             0.015000
last_0_mo_orderQ              0.014788
4_wk_orderQ_slope             0.013690
postal_code                   0.011950
2_wk_orderQ_slope             0.011567
last_6_mo_orderQ_slope_6      0.010764
last_8_mo_orderQ              0.010071
2_mo_orderQ

#### RandomizedSearchCV

In [321]:
# 設定要訓練的超參數組合
n_estimators = [50,100, 150, 200]
max_depth = [3,5,7,9,11]
param_grid = dict( n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
random_search = RandomizedSearchCV(gbc , param_grid, n_jobs=-1, verbose=1)
##開始搜尋最佳參數
random_result = random_search.fit(x_train, y_train)
print("Best Accuracy: %f using %s" % (random_result.best_score_, random_result.best_params_))

C:\Users\s84120703\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  9.1min finished


Best Accuracy: 0.773785 using {'n_estimators': 150, 'max_depth': 3}


In [322]:
# 使用最佳參數重新建立模型
gbc_bestparam = GradientBoostingClassifier(max_depth=random_result.best_params_['max_depth'],
                                           n_estimators=random_result.best_params_['n_estimators'])
gbc_bestparam.fit(x_train, y_train)
# 預測測試集
gbc_bestparam_y_pred = gbc_bestparam.predict(x_test)

print("confusion matrix：\n", confusion_matrix(y_test,gbc_bestparam_y_pred))
print("metrics：\n", classification_report(y_test,gbc_bestparam_y_pred))
print("feature importance:\n",pd.Series(gbc_bestparam.feature_importances_,x_columns).sort_values(ascending=False))


confusion matrix：
 [[2180  223]
 [ 570  451]]
metrics：
               precision    recall  f1-score   support

           0       0.79      0.91      0.85      2403
           1       0.67      0.44      0.53      1021

    accuracy                           0.77      3424
   macro avg       0.73      0.67      0.69      3424
weighted avg       0.76      0.77      0.75      3424

feature importance:
 last_0_3_mo_orderQ            2.149908e-01
last_1_6_wk_orderQ            1.553861e-01
last_2_mo_orderQ              6.180498e-02
last_1_mo_orderQ              5.600181e-02
last_0_6_mo_orderQ            5.098821e-02
last_7_12_wk_orderQ           3.716690e-02
last_4_mo_orderQ              3.240186e-02
net_volume                    2.027047e-02
4_wk_orderQ_slope             1.644785e-02
last_0_mo_orderQ              1.567096e-02
3_mo_orderQ_slope             1.400653e-02
postal_code                   1.296877e-02
2_wk_orderQ_slope             1.227380e-02
last_6_mo_orderQ_slope_6      1.14202

In [324]:
auc = roc_auc_score(y_test, gbc_y_predict) 
print("AUC: ", auc) 

AUC:  0.6759916493544024


#### SMOTE  over sampling : 我最後是用這個model

In [328]:
gbc_over = GradientBoostingClassifier(random_state=1) 
gbc_over.fit(over_samples_x, over_samples_y)
gbc_over_y_predict = gbc_over.predict(x_test)

In [332]:
auc = roc_auc_score(y_test, gbc_over_y_predict) # 使用 roc_auc_score 來評估。 **這邊特別注意 y_pred 必須要放機率值進去!**
print("SMOTE AUC: ", auc) # 得到結果約 0.5，與亂猜的結果相近，因為我們的預測值是用隨機生成的

print("confusion matrix：\n", confusion_matrix(y_test,gbc_over_y_predict))
print("metrics：\n", classification_report(y_test,gbc_over_y_predict))
print("feature importance:\n",pd.Series(gbc_over.feature_importances_,x_columns).sort_values(ascending=False))

SMOTE AUC:  0.7109428591342115
confusion matrix：
 [[2007  396]
 [ 422  599]]
metrics：
               precision    recall  f1-score   support

           0       0.83      0.84      0.83      2403
           1       0.60      0.59      0.59      1021

    accuracy                           0.76      3424
   macro avg       0.71      0.71      0.71      3424
weighted avg       0.76      0.76      0.76      3424

feature importance:
 last_0_3_mo_orderQ            0.229926
last_1_mo_orderQ              0.186423
last_2_mo_orderQ              0.094835
last_4_mo_orderQ              0.071314
last_7_12_wk_orderQ           0.052171
last_3_mo_orderQ              0.038612
last_9_mo_orderQ              0.030543
last_5_mo_orderQ              0.028707
last_13_18_wk_orderQ          0.020829
last_1_6_wk_orderQ            0.015076
last_0_mo_orderQ              0.013153
last_6_mo_orderQ              0.012117
last_14_mo_orderQ             0.011961
last_18_mo_orderQ             0.011471
product_line       

### predict to test dataset

In [335]:
test_x_all.head(2)

,sequence_id,site_id,item,item_type,item_ownership,product_line,customer_class_code,city,postal_code,state,...,last_15_mo_cancelQ_slope_3,last_18_mo_cancelQ_slope_3,last_21_mo_cancelQ_slope_3,last_24_mo_cancelQ_slope_3,ordered_quantity,canceled_quantity,net_volume,canceled_ratio,isOrdered,isCanceled
13694,140365A0464474,557,88,1,2,1,1,784,3563,16,...,0.0,0.0,0.0,0.0,25.0,0,160.50,0.0,1,0
13695,140365A0464909,557,270,1,2,10,1,784,3563,16,...,0.0,0.0,0.0,0.0,21.0,0,144.56,0.0,1,0


In [336]:
test_x = test_x_all.drop(['sequence_id'],axis =1).values

In [337]:
test_p = gbc_over.predict(test_x)
test_p

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [338]:
result = pd.DataFrame()
result['sequence_id'] = test_x_all['sequence_id'] 
result['prediction']= test_p
result.to_csv('result.csv')

In [339]:
test_proba = gbc_over.predict_proba(test_x)

In [344]:
result_p = pd.DataFrame()
result_p['sequence_id'] = test_x_all['sequence_id'] 
result_p['prediction']= test_proba[:,1]
result_p.to_csv('result_p_v0.csv')

In [345]:
test_x.shape

(5870, 152)